In [1]:
import pandas as pd

DATA_ROOT = "../dataset/ml1m/"
rating_path = DATA_ROOT + "ratings.dat"

In [2]:
ratings = pd.read_table(rating_path, sep = "::", encoding = "ISO-8859-1",
                        header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])

/tmp/ipykernel_8996/528942237.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(rating_path, sep = "::", encoding = "ISO-8859-1",


In [3]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
clicks = ratings['rating'] >= 3
likes = ratings['rating'] >= 4
stars = ratings['rating'] == 5
print(sum(clicks), sum(likes), sum(stars))

836478 575281 226310


In [6]:
import numpy as np
ratings['is_click'] = np.array(clicks).astype(int)
ratings['is_like'] = np.array(likes).astype(int)
ratings['is_star'] = np.array(stars).astype(int)
ratings = ratings.drop(['rating'], axis = 1)
ratings[:5]

,user_id,movie_id,timestamp,is_click,is_like,is_star
0,1,1193,978300760,1,1,1
1,1,661,978302109,1,0,0
2,1,914,978301968,1,0,0
3,1,3408,978300275,1,1,0
4,1,2355,978824291,1,1,1


In [7]:
from utils import run_multicore
filterred_df = run_multicore(ratings, user_key = "user_id", item_key = "movie_id", 
                             n_core = 20, auto_core = False, filter_rate = 0.2)

/opt/conda/envs/pt_rlsim/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Filter 20-core data.
n_core = 20
N-core is set to [5,100]
Filtering 20-core data
Iteration 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000209/1000209 [00:00<00:00, 1177276.65it/s]


Number of removed record: 4867
Iteration 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 995342/995342 [00:00<00:00, 1611811.91it/s]


Number of removed record: 188
Iteration 3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 995154/995154 [00:00<00:00, 1621639.48it/s]


Number of removed record: 0
Size change: 1000209 --> 995154


In [8]:
users = list(ratings['user_id'].unique())
items = list(ratings['movie_id'].unique())
print(len(users), len(items), len(ratings))
users = list(filterred_df['user_id'].unique())
items = list(filterred_df['movie_id'].unique())
print(len(users), len(items), len(filterred_df))

6040 3706 1000209
6022 3043 995154


In [9]:
filterred_df = filterred_df.sort_values(by = ['user_id', 'timestamp'])
filterred_df[:5]

,user_id,movie_id,timestamp,is_click,is_like,is_star
31,1,3186,978300019,1,1,0
22,1,1270,978300055,1,1,1
27,1,1721,978300055,1,1,0
37,1,1022,978300055,1,1,1
24,1,2340,978300103,1,0,0


In [10]:
rating_save_path = DATA_ROOT + "log_session.csv"
filterred_df.to_csv(rating_save_path, index = False)

### Item/User Meta Data

In [15]:
import pandas as pd
item_meta = pd.read_table(DATA_ROOT + "movies.dat", sep = "::", encoding = "ISO-8859-1",
                           header=None, names=['movie_id', 'title', 'genres'])
item_meta[:5]

/tmp/ipykernel_77110/185394067.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  item_meta = pd.read_table(DATA_ROOT + "movies.dat", sep = "::", encoding = "ISO-8859-1",


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
item_meta = item_meta.drop(["title"], axis = 1)
genre = [g.replace('\'s','').replace('-','').replace('|',',') if len(g)>0 else 0 for g in item_meta["genres"]]
item_meta = item_meta.drop(["genres"], axis = 1)
item_meta.insert(1, "genres", genre)

In [17]:
item_meta.to_csv(DATA_ROOT + "movies_processed.csv", index = False)
item_meta[:5]

,movie_id,genres
0,1,"Animation,Children,Comedy"
1,2,"Adventure,Children,Fantasy"
2,3,"Comedy,Romance"
3,4,"Comedy,Drama"
4,5,Comedy


In [18]:
import pandas as pd
user_meta = pd.read_table(DATA_ROOT + "users.dat", sep = "::", encoding = "ISO-8859-1",
                          header=None, names=['user_id', 'gender', 'age', 'occupation', 'zip_code'])
user_meta[:5]

/tmp/ipykernel_77110/3631703635.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_meta = pd.read_table(DATA_ROOT + "users.dat", sep = "::", encoding = "ISO-8859-1",


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [19]:
user_meta = user_meta.drop(["occupation", 'zip_code'], axis = 1)
user_meta.to_csv(DATA_ROOT + "users_processed.csv", index = False)
user_meta[:5]

,user_id,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25
